In [3]:
!pip -q install efficientnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.4 MB/s eta 0:00:00


In [2]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [4]:
# Cell 2: Import libraries
import numpy as np
import pandas as pd
import os
import random

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import efficientnet.tfkeras as efn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [4]:
# Cell 3: Load the training data
train = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/train.csv')
train['path'] = '/kaggle/input/cassava-leaf-disease-classification/train_images/' + train['image_id']
train['label'] = train['label'].astype(str)

In [5]:
# Cell 4: Split into train and validation sets
train_df, val_df = train_test_split(train, test_size=0.2, stratify=train['label'], random_state=seed)

In [6]:
# Cell 5: Define image size and batch size
IMG_SIZE = 380 
BATCH_SIZE = 16 

In [7]:
# Cell 6: Data generators with augmentation for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)  # No augmentation for validation

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col='path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col='path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 17117 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


In [8]:
# Cell 7: Build the model using EfficientNetB4 with transfer learning
base_model = efn.EfficientNetB4(weights='noisy-student', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = True  # Fine-tune the base model

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

I0000 00:00:1763634769.340198      89 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


71678424/71678424 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b4 (Functional)    │ (None, 12, 12, 1792)   │    17,673,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         8,965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,682,781 (67.45 MB)

 Trainable params: 17,557,581 (66.98 MB)

 Non-trainable params: 125,200 (489.06 KB)

In [9]:
# Cell 8: Define callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [10]:
# Cell 9: Train the model
EPOCHS = 20

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stopping, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20


I0000 00:00:1763634880.459880     158 service.cc:148] XLA service 0x7c21000041a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763634880.460929     158 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1763634889.923475     158 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1763634906.318742     158 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763634906.505841     158 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763634906.877892     158 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763634907.0653

 312/1070 ━━━━━━━━━━━━━━━━━━━━ 8:20 660ms/step - accuracy: 0.6328 - loss: 1.0291

E0000 00:00:1763635174.059830     159 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763635174.246343     159 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763635174.612199     159 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763635174.799443     159 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1763635175.280476     159 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

1070/1070 ━━━━━━━━━━━━━━━━━━━━ 1001s 777ms/step - accuracy: 0.7104 - loss: 0.8100 - val_accuracy: 0.8332 - val_loss: 0.4789 - learning_rate: 1.0000e-04
Epoch 2/20
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 656s 613ms/step - accuracy: 0.8355 - loss: 0.4657 - val_accuracy: 0.8612 - val_loss: 0.4143 - learning_rate: 1.0000e-04
Epoch 3/20
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 637s 595ms/step - accuracy: 0.8566 - loss: 0.4090 - val_accuracy: 0.8694 - val_loss: 0.4025 - learning_rate: 1.0000e-04
Epoch 4/20
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 631s 589ms/step - accuracy: 0.8730 - loss: 0.3679 - val_accuracy: 0.8696 - val_loss: 0.3840 - learning_rate: 1.0000e-04
Epoch 5/20
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 635s 594ms/step - accuracy: 0.8774 - loss: 0.3546 - val_accuracy: 0.8685 - val_loss: 0.3892 - learning_rate: 1.0000e-04
Epoch 6/20
1070/1070 ━━━━━━━━━━━━━━━━━━━━ 838s 783ms/step - accuracy: 0.8852 - loss: 0.3301 - val_accuracy: 0.8755 - val_loss: 0.3720 - learning_rate: 1.0000e-04
Epoch 7/20
1070/1070 ━━━━━━━━━━━━━━━━━

In [11]:
# Cell 10: Prepare test data
test_dir = '/kaggle/input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(test_dir)
test_df = pd.DataFrame(test_images, columns=['image_id'])
test_df['path'] = test_dir + test_df['image_id']

test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col=None,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)

Found 1 validated image filenames.


In [12]:
# Cell 11: Make predictions on test set
preds = model.predict(test_gen, verbose=1)
labels = np.argmax(preds, axis=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step


In [5]:
# Cell 12: Generate submission file
submission = pd.DataFrame({'image_id': test_images, 'label': labels})
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully!")

NameError: name 'test_images' is not defined